In [1]:
import sys ,glob, os,pickle,random
import numpy as np
import datetime as dt
sys.path.append('/home/giacomo/Documents/Denoiser_GPS/sharing_gratsid_tf_in_development')
sys.path.append('/home/giacomo/Documents/Step_model')
sys.path.append('/home/giacomo/Documents/Denoiser_GPS/Denoiser_code')
from funcs_4_DL_resids import *
from gratsid_tf_gpu_functions_SHARED import *


2023-07-10 16:25:22.779868: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 16:25:22.847741: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 16:25:22.849905: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-10 16:25:22.849913: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

Num GPUs Available:  0


2023-07-10 16:25:23.756949: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-10 16:25:23.756973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: giacomo-Z590-AORUS-MASTER
2023-07-10 16:25:23.756976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: giacomo-Z590-AORUS-MASTER
2023-07-10 16:25:23.757018: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.116.4
2023-07-10 16:25:23.757029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.116.4
2023-07-10 16:25:23.757031: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.116.4


In [19]:
# components=['E/' ] #'E/','N/' #'U/'
components=['U/']
soln_folder_path='/home/giacomo/Documents/Denoiser_GPS/Wordwide_dataset/sols_tables_'
data_folder_path='/home/giacomo/Documents/Denoiser_GPS/Wordwide_dataset/data_bank/'
names=id_names_npz(soln_folder_path+components[0])
gen_jjj = np.vectorize(lambda x,y,z: dt.date.toordinal(dt.date(x,y,z)))
name=names[0]

sa_coeff=0
tr_coeff=0
trans_coeff=0

for c in range(len(components)):

    ############ finally build ############
    soln_folder=soln_folder_path+components[c]
    sol_path = soln_folder+name+'.npz'
    options = np.load(sol_path,allow_pickle=True)['options']
    options = options.item()
    data_cols = np.load(sol_path)['data_cols']
    
    ten_step=10
    ########## Allocate  ##########
    for i in range(len(names)):
        name=names[i]
        sol_path = soln_folder+name+'.npz'
        data_path = data_folder_path+name+'.txt'
    
        ## loading in the data and the corresponding gratsid solution table
        data = np.loadtxt(data_path)
        perm = list(np.load(sol_path,allow_pickle=True)['perm'])
        sols = list(np.load(sol_path,allow_pickle=True)['sols'])
    
        ## converting time to python datetime integer and isolating the fit directional components
        t = gen_jjj(data[:,0].astype(int),data[:,1].astype(int),data[:,2].astype(int))
        y = data[:,data_cols] ## columns 3,4,5  (in python indexing) are E,N,U
        
        signal = fit_decompose(t,y,None,options['tik_mul'], \
                sols,np.asarray(perm),options['bigTs'],options['Fs']) 
        
        table = np.vstack([perm,sols[-1][-1]])
        G, m_keys = assemble_G_return_keys(t,table,options['bigTs'],options['Fs'])

        residual,residual_val,weighted_residual_val,m \
            = single_fit_predict_tf(G_in=G,y_in=y,err_in=None,tik_mul=options['tik_mul'])
        
        ## This is the maximum coefficient for seasonal a annual amplitudes
        if max(m[np.where(m_keys==2)])>sa_coeff:
            sa_coeff=max(m[np.where(m_keys==2)])
            
        if max(m[np.where(m_keys==1)])>tr_coeff:
            tr_coeff=max(m[np.where(m_keys==1)])
        
        if 3 in m_keys:
            if max(m[np.where(m_keys==3)])>trans_coeff:
                trans_coeff=max(m[np.where(m_keys==3)])
            
        perc=i*100/len(names)
        if perc>ten_step:
            print(str(round(perc))+'%')
            ten_step+=10
            print(sa_coeff)
            
print('Max amplitude for seasonals: ',sa_coeff)
print('Max amplitude for transients: ',trans_coeff)
print('Max amplitude for trend: ',tr_coeff)

10%
[0.18801744]
20%
[0.22452118]
30%
[0.22452118]
40%
[0.28170884]
50%
[0.28170884]
60%
[0.32102071]
70%
[0.32102071]
80%
[0.32102071]
90%
[2.90863324]
Max amplitude for seasonals:  [2.90863324]
Max amplitude for transients:  [30.30877162]
Max amplitude for trend:  [59.60290465]


In [18]:
tr_coeff

array([1795.5096329])